In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 527 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 881.9 MB 22 kB/s s eta 0:00:01    |███▎                            | 89.5 MB 11.2 MB/s eta 0:01:11     |██████████▉                     | 299.6 MB 10.6 MB/s eta 0:00:55     |███████████████████████▉        | 658.1 MB 10.3 MB/s eta 0:00:22     |████████████████████████▎       | 669.0 MB 10.5 MB/s eta 0:00:21
     |████████████████████████████████| 409 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 840 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 603 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 4.8 MB/s  eta 0:00:01
     |███████████████████████████

     |████████████████████████████████| 191 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 270 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 952 kB 11.5 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=494240 sha256=eb202431e6b05967847675e2d181025c9f7caef81c093336964c42731fae1575
  Stored in directory: /root/.cache/pip/wheels/63/f1/0c/e56d12b3804345ce5ba34279cbfe583ecafdd1401551457330
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3949 sha256=8cf6a6fe23e35ccf617578060d3190edbc94cc6f89742bcf984fa925f8e630b9
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built future idna-ssl
  Attempting uninstall: setuptools
    Found existing installation: setuptools 51.1.1
    Uninstalling setuptools-51.1.1:
      Successfully uninstalled setuptools-51.1.1

# Run code with setup

In [ ]:
"GRU", "LSTM", "GCN", "TGCN","TGCN_LSTM","TGCN_UGRNN"

In [ ]:
["mse_with_regularizer","mse_with_regularizer_l1"]

In [ ]:
"losloop","PEMS08","PEMS04",""

In [ ]:
from subprocess import Popen,PIPE
model = ["TGCN"]
max_epochs = "1"
learning_rate = "0.001" 
weight_decay ="0"
enable_progress_bar = "False"
batch_size = "64"
hidden_dim = "64"
cell_dim = "64"
dropout = "0"
loss = ["mse"]
data = ["losloop"]
gpus = "0"
for i in data:
    for j in loss:
        for k in model:
            for l in layer_2:
                print("Dataset:",i," Loss:",j," Model:",k,"Time:",l)
                cmd_str = ["python",
                       "main.py",
                       "--enable_progress_bar",enable_progress_bar,
                       "--model_name",k,
                       "--max_epochs",max_epochs,
                       "--learning_rate",learning_rate,
                       "--weight_decay",weight_decay,
                       "--batch_size",batch_size,
                       "--hidden_dim",hidden_dim,
                       "--cell_dim",cell_dim,
                       "--dropout",dropout,
                       "--loss",j,
                       "--layer_2",l,
                       "--data",i,
                       "--gpus",gpus,
                      ]
                p = Popen(cmd_str,stdout=PIPE)
                for m in p.stdout:
                    print(m)

# Test one with time 

In [ ]:
import time

In [2]:
%%time

!python main.py \
--model_name TGCN \
--data shenzhen \
--max_epochs 10 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 32 \
--hidden_dim 64 \
--dropout 0.2 \
--svd 0 \
--loss "mse" \
--seq_len 4 \
--pre_len 4

[2023-05-24 08:41:39,081 INFO]{'logger': True, 'checkpoint_callback': None, 'enable_checkpointing': True, 'default_root_dir': None, 'gradient_clip_val': None, 'gradient_clip_algorithm': None, 'process_position': 0, 'num_nodes': 1, 'num_processes': 1, 'devices': None, 'gpus': None, 'auto_select_gpus': False, 'tpu_cores': None, 'ipus': None, 'log_gpu_memory': None, 'progress_bar_refresh_rate': None, 'enable_progress_bar': True, 'overfit_batches': 0.0, 'track_grad_norm': -1, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'accumulate_grad_batches': None, 'max_epochs': 10, 'min_epochs': None, 'max_steps': -1, 'min_steps': None, 'max_time': None, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0, 'limit_test_batches': 1.0, 'limit_predict_batches': 1.0, 'val_check_interval': 1.0, 'flush_logs_every_n_steps': None, 'log_every_n_steps': 50, 'accelerator': None, 'strategy': None, 'sync_batchnorm': False, 'precision': 32, 'enable_model_summary': True, 'weights_summary': 'top', 'weights_sa

# Results to save

In [ ]:
DATALOADER:0 VALIDATE RESULTS time 4 mse sz 500epoch original
MAE:2.6897
R2:0.8591
RMSE:3.8249
ACCURACY:0.7301
DATALOADER:0 VALIDATE RESULTS time 3 mse sz 500epoch original
{'ExplainedVar': 0.8580623865127563,
 'MAE': 2.7100107669830322,
 'R2': 0.857498049736023,
 'RMSE': 3.8475542068481445,
 'accuracy': 0.7285716533660889,
 'val_loss': 14.803674697875977}
DATALOADER:0 VALIDATE RESULTS time 2 mse sz 500epoch original
{'ExplainedVar': 0.863174557685852,
 'MAE': 2.6422271728515625,
 'R2': 0.8630218505859375,
 'RMSE': 3.771233558654785,
 'accuracy': 0.7339633107185364,
 'val_loss': 14.22220230102539}
DATALOADER:0 VALIDATE RESULTS time 1 mse sz 500epoch original
{'ExplainedVar': 0.869576632976532,
 'MAE': 2.560318946838379,
 'R2': 0.8695765733718872,
 'RMSE': 3.6794581413269043,
 'accuracy': 0.7404475212097168,
 'val_loss': 13.538414001464844}

In [ ]:
DATALOADER:0 VALIDATE RESULTS 4 dropout sz 500 epochs original
{'ExplainedVar': 0.8653743267059326,
 'MAE': 2.578397512435913,
 'R2': 0.8652923107147217,
 'RMSE': 3.740065336227417,
 'accuracy': 0.7361570596694946,
 'val_loss': 13.988088607788086}

In [ ]:
DATALOADER:0 VALIDATE RESULTS time 1 l1 sz 500 epochs original
{'ExplainedVar': 0.6901556253433228,
 'MAE': 4.370548248291016,
 'R2': 0.6900507211685181,
 'RMSE': 5.672628402709961,
 'accuracy': 0.5998473763465881,
 'val_loss': 32.185523986816406}
DATALOADER:0 VALIDATE RESULTS time 4 l1 sz 500 epochs original
{'ExplainedVar': 0.0,
 'MAE': 8.766261100769043,
 'R2': -1.1920928955078125e-07,
 'RMSE': 10.565918922424316,
 'accuracy': 0.26397448778152466,
 'val_loss': 111.641845703125}
DATALOADER:0 VALIDATE RESULTS time 4 mse losloop 500 epochs original
{'ExplainedVar': 0.5805838108062744,
 'MAE': 6.201210021972656,
 'R2': 0.5803792476654053,
 'RMSE': 8.951526641845703,
 'accuracy': 0.847675621509552,
 'val_loss': 80.12982177734375}
DATALOADER:0 VALIDATE RESULTS time 4 dropout losloop 500 epochs original
{'ExplainedVar': 0.5526901483535767,
 'MAE': 6.336508274078369,
 'R2': 0.5525348782539368,
 'RMSE': 9.243420600891113,
 'accuracy': 0.8427085280418396,
 'val_loss': 85.44081115722656}
